In [39]:
import pandas as pd
from pandas_datareader import data, wb
import datetime

In [40]:
start = pd.to_datetime('2015-09-24')
end = pd.to_datetime('2020-09-24')
data_df = data.DataReader('AAPL','yahoo',start,end)

In [41]:
df1=data_df.reset_index()['Close']
df1

0        28.750000
1        28.677500
2        28.110001
3        27.264999
4        27.575001
           ...    
1255    106.839996
1256    110.080002
1257    111.809998
1258    107.120003
1259    108.220001
Name: Close, Length: 1260, dtype: float64

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
import numpy as np
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
df1

array([[0.05524442],
       [0.05459475],
       [0.0495094 ],
       ...,
       [0.79954302],
       [0.75751609],
       [0.76737315]])

In [43]:
training_size=int(len(df1)*0.80)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [44]:
print(test_data.shape)
len(test_data)

(252, 1)


252

In [45]:
x_input=test_data[152:].reshape(1,-1)
x_input.shape

(1, 100)

In [46]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
temp_input

[0.46422333689000306,
 0.47110090197025656,
 0.47806802758108835,
 0.4923832187083882,
 0.5033156197188771,
 0.4952507281236953,
 0.486827389305748,
 0.4910614830307749,
 0.4869617984684329,
 0.5032035665054282,
 0.49912637462595666,
 0.5127694515730146,
 0.507437660230455,
 0.5120077768621942,
 0.5071688419050852,
 0.5102603210136557,
 0.5105739879711321,
 0.5098795178416542,
 0.5186388795663138,
 0.5219768210510182,
 0.5259644523999112,
 0.5196917969185572,
 0.5402572192111578,
 0.5446480780481056,
 0.5682378432347515,
 0.5880640151325849,
 0.55011427855335,
 0.5566109720947846,
 0.5659975993675797,
 0.5863614079161528,
 0.5852637102986201,
 0.5855773772560965,
 0.5810744652056997,
 0.6015726829169579,
 0.6187329591435116,
 0.6042385785882376,
 0.6149469415386459,
 0.5898338269303594,
 0.6080918007744457,
 0.6148573126412502,
 0.6133115389035563,
 0.6133115389035563,
 0.6351315606592449,
 0.6325328695693075,
 0.6519781699283222,
 0.6556522026865562,
 0.6571531277921436,
 0.6531879207

In [47]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('my_model1.h5')
# summarize model.
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 60)           14880     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 60)           29040     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60)                29040     
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 73,021
Trainable params: 73,021
Non-trainable params: 0
_________________________________________________________________


In [48]:
from numpy import array
lst_output=[]
n_steps=100
i=0
while(i<11):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        #yhat = scaler.inverse_transform(yhat)
        #print("{} day Prediction {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        #print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        #print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    



In [49]:
l1 = lst_output[1:11]
#print(l1)
#print(len(l1))

In [50]:
import itertools
b = l1
c =list(itertools.chain.from_iterable(b))
#print(c)


In [51]:
e = scaler.inverse_transform(b)


In [52]:
f = list(itertools.chain.from_iterable(e))
#print(f)

In [53]:
for i in range(10):
  print("Day {} Prediction : {}".format(i+1,f[i].round(2)))

Day 1 Prediction : 94.56
Day 2 Prediction : 90.22
Day 3 Prediction : 85.45
Day 4 Prediction : 80.78
Day 5 Prediction : 76.48
Day 6 Prediction : 72.67
Day 7 Prediction : 69.41
Day 8 Prediction : 66.69
Day 9 Prediction : 64.49
Day 10 Prediction : 62.74
